# Project 12: Transformer Architecture from Scratch

## Goal
Build a tiny decoder-only transformer from first principles.

## Learning Objectives
- Multi-head self-attention mechanism
- Feed-forward networks
- Positional embeddings
- Stacking transformer blocks

## Model Configuration
- Vocabulary size: 5000 tokens
- Hidden dimension: 384
- Attention heads: 6
- Layers: 4-6
- Total parameters: ~20-50M

In [ ]:
# Setup
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {torch.device('mps' if torch.backends.mps.is_available() else 'cpu')}")